In [1]:
import tensorflow as tf
import os
import numpy as np


In [3]:
import sys

In [4]:
# Split text into words
python3 = sys.version_info[0] == 3


In [5]:
python3

True

In [7]:
import string

In [8]:
with open('lovecraft.txt', 'r') as f:
    input_str = f.read().lower()
    if python3:
        trans = input_str.maketrans('', '', string.punctuation)
        input_str = input_str.translate(trans)
    else:
        input_str = input_str.decode('utf-8').translate(None, string.punctuation)
    words = input_str.split()
    num_words = len(words)
        
    
    
    
    

In [10]:
import collections

In [11]:
# Convert words to values
word_freq = collections.Counter(words).most_common()
vocab_size = len(word_freq)
lookup = dict()
for word, _ in word_freq:
    lookup[word] = len(lookup)
input_vals = np.asarray([[lookup[str(word)]] for word in words])
input_vals = input_vals.reshape(-1, )
    
    

In [12]:
# Set values 
input_size = 6
batch_size = 10
num_hidden = 600
    
    

In [13]:
# Placeholder 
input_holder = tf.placeholder(tf.float32, [batch_size, input_size])
label_holder = tf.placeholder(tf.float32, [batch_size, vocab_size])
    
    

In [15]:
# Reshape input and feed to RNN
cell = tf.nn.rnn_cell.BasicRNNCell(num_hidden)
outputs, _ = tf.nn.static_rnn(cell, [input_holder], dtype=tf.float32)
    
    

In [21]:
# Compute loss 
weights = tf.Variable(tf.random_normal([num_hidden, vocab_size]))

biases = tf.Variable(tf.random_normal([vocab_size]))

In [22]:
model = tf.matmul(outputs[-1], weights) + biases
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=label_holder))

In [23]:
# Create optimizer and check result
optimizer = tf.train.AdagradOptimizer(0.1).minimize(loss)
check = tf.equal(tf.argmax(model, 1), tf.argmax(label_holder, 1))
correct = tf.reduce_sum(tf.cast(check, tf.float32))

In [25]:
import time

In [26]:
# Execute the graph
start_time = time.time()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    input_block = np.empty([batch_size, input_size])
    label_block = np.empty([batch_size, vocab_size])
    
    step = 0
    num_correct = 0.
    accuracy = 0.
    while accuracy < 95:
        
        for i in range(batch_size):
            
            offset = np.random.randint(num_words - (input_size+1))
            input_block[i, :] = input_vals[offset:offset+input_size]
            label_block[i, :] = np.eye(vocab_size) [input_vals[offset+input_size]]
        _, corr = sess.run([optimizer, correct],
                          feed_dict = {input_holder: input_block, label_holder: label_block})
        num_correct += corr
        accuracy = 100 * num_correct/(1000*batch_size)
        if step % 1000 == 0:
            print('Step', step, '-Accuracy =', accuracy)
            num_correct = 0
        step += 1
        
        
# Display timing result
duration = time.time() - start_time
print('Time to reach 95% accuracy: {: .2f} seconds'.format(duration))

Step 0 -Accuracy = 0.0
Step 1000 -Accuracy = 8.63
Step 2000 -Accuracy = 30.24
Step 3000 -Accuracy = 53.45
Step 4000 -Accuracy = 69.0
Step 5000 -Accuracy = 77.29
Step 6000 -Accuracy = 82.46
Step 7000 -Accuracy = 84.44
Step 8000 -Accuracy = 87.85
Step 9000 -Accuracy = 89.06
Step 10000 -Accuracy = 90.18
Step 11000 -Accuracy = 91.75
Step 12000 -Accuracy = 92.32
Step 13000 -Accuracy = 93.95
Step 14000 -Accuracy = 94.04
Step 15000 -Accuracy = 94.64
Step 16000 -Accuracy = 95.02
Time to reach 95% accuracy:  110.20 seconds
